In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/fridge/fridge.csv')
df.head()

,price,title,added_time,metro,seller_name,seller_rating,link,address,description,dist_to_metro,parsed_at,seller_review,seller_status,views
0,5500.0,Холодильник,вчера в 23:06,Озерки,Магазин Б/У Техника,NaN,https://www.avito.ru/sankt-peterburg/bytovaya_...,Санкт-Петербург,"Холодильник lg Гарантия 3 месяца, доставка, во...",NaN,2020-03-31 01:15:33.366149,0.0,Компания,1218.0
1,6200.0,Холодильник бу Nord,вчера в 18:39,Улица Дыбенко,"""ТехноПлюс"" БЫТОВАЯ ТЕХНИКА БУ",3.3,https://www.avito.ru/sankt-peterburg/bytovaya_...,"Санкт-Петербург, ул. Дыбенко, 26","Холодильник Nord/Норд - В/Ш/Г(142/60-56) ""Техн...",NaN,2020-03-31 01:15:34.915264,4.0,Компания,6.0
2,6400.0,Холодильник,вчера в 18:09,Академическая,"""ТехноПлюс"" БЫТОВАЯ ТЕХНИКА БУ",3.3,https://www.avito.ru/sankt-peterburg/bytovaya_...,"Санкт-Петербург, Калининский район, муниципаль...",Холодильник Атлант - В/Ш/Г(146/47/60) “ТехноПл...,NaN,2020-03-31 01:15:36.435182,4.0,Компания,50.0
3,5900.0,Холодильник,вчера в 15:10,Приморская,"""ТехноПлюс"" БЫТОВАЯ ТЕХНИКА БУ",3.3,https://www.avito.ru/sankt-peterburg/bytovaya_...,"Санкт-Петербург, ул. Одоевского, 29",Холодильник Позис - В/Ш/Г(185/60/57) “ТехноПлю...,NaN,2020-03-31 01:15:38.122599,4.0,Компания,10.0
4,1000.0,Холодильник,вчера в 22:57,Беговая,Ирья,5.0,https://www.avito.ru/sankt-peterburg/bytovaya_...,"Санкт-Петербург, Богатырский пр-т, 58к3",Продам холодильник. Работает только морозилка....,"1,9 км",2020-03-31 01:15:40.715788,1.0,Частное лицо,75.0


Переведем столбец ```parsed_at``` в *datetime-object*

In [3]:
df['parsed_at'] = pd.to_datetime(df['parsed_at'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952 entries, 0 to 1951
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   price          1951 non-null   float64       
 1   title          1951 non-null   object        
 2   added_time     1951 non-null   object        
 3   metro          1758 non-null   object        
 4   seller_name    1951 non-null   object        
 5   seller_rating  1081 non-null   float64       
 6   link           1952 non-null   object        
 7   address        1951 non-null   object        
 8   description    1951 non-null   object        
 9   dist_to_metro  1261 non-null   object        
 10  parsed_at      1952 non-null   datetime64[ns]
 11  seller_review  1952 non-null   float64       
 12  seller_status  1951 non-null   object        
 13  views          1951 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(9)
memory usage: 213.6+ KB


# Посмотрим на NaN

In [4]:
df.isna().sum()

price              1
title              1
added_time         1
metro            194
seller_name        1
seller_rating    871
link               0
address            1
description        1
dist_to_metro    691
parsed_at          0
seller_review      0
seller_status      1
views              1
dtype: int64

Удалим все записи, в которых нет столбца added_time

In [5]:
df = df[df['added_time'].notna()]

In [6]:
df.isna().sum()

price              0
title              0
added_time         0
metro            193
seller_name        0
seller_rating    870
link               0
address            0
description        0
dist_to_metro    690
parsed_at          0
seller_review      0
seller_status      0
views              0
dtype: int64

Теперь NaN-ов в важных столбцах нет. Ближайшее метро и расстояние до него можно будет получить через API Яндекса, а на рейтинг продавца можно не обращать особого внимания.

# Избавимся от объявлений, продающих неисправные холодильники или запчасти для них 
Профильтруем датафрейм по списку слов в заголовке. Потом можно будет сделать более продвинутый алгоритм, который будет работать с описанием товара.  
Функция будет принимать текстовый файл со списком запрещенных слов.

In [7]:
def filter_df(df, filter_file):
    with open(filter_file, 'r', encoding='utf-8') as f:
        filter_words = f.readlines()
    return df[~df.title.str.lower().str.contains('|'.join(filter_words))]

In [8]:
df = filter_df(df, 'data/fridge/fridge_filter.txt')
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
price,1939.0,3896.818979,1711.678637,750.0,2500.0,4000.0,5100.0,6500.0
seller_rating,1070.0,4.464393,0.821742,1.0,4.0,5.0,5.0,5.0
seller_review,1939.0,4.111913,6.962627,0.0,0.0,1.0,8.0,95.0
views,1939.0,56.066013,351.849745,1.0,10.0,22.0,45.5,14783.0


# Переведем строки в более удобные форматы
## Переведем столбец *dist_to_metro*

In [9]:
def dist_to_float(x):
    if type(x) != str:
        return x
    dist, unit = x.split()
    dist = dist.replace(',', '.')
    dist = float(dist)
    if unit == 'км':
        dist *= 1000
    return dist
df['dist_to_metro'] = df['dist_to_metro'].map(dist_to_float)

In [10]:
df.dist_to_metro.describe()

count     1251.000000
mean      1094.884093
std        929.183408
min        100.000000
25%        500.000000
50%        900.000000
75%       1400.000000
max      10200.000000
Name: dist_to_metro, dtype: float64

## Переведем столбец *added_time*

In [11]:
df.added_time.str.contains(' в ').value_counts()

True    1939
Name: added_time, dtype: int64

Каждая запись имеет ```' в '``` внутри. Значит, мы сможем безопасно разделить стринг по этому токену

In [12]:
from datetime import datetime, timedelta

def str_to_datetime(string_date, parsed_at):
    date, time = string_date.split(' в ')
    hour, minute = list(map(int, time.split(':')))
    if date == 'вчера':
        yesterday = parsed_at - timedelta(days=1)
        day = yesterday.day
        month = yesterday.month
    elif date == 'сегодня':
        day = parsed_at.day
        month = parsed_at.month
    else:
        ru_to_num = {
            'января':1,
            'февраля':2,
            'марта':3,
            'апреля':4,
            'мая':5,
            'июня':6,
            'июля':7,
            'августа':8,
            'сентября':9,
            'октября':10,
            'ноября':11,
            'декабря':12,
        }
        day, month = date.split()
        day = int(day)
        month = ru_to_num[month]
    return datetime(2020, month, day, hour, minute) 

In [13]:
df['added_time'] = df.apply(lambda x: str_to_datetime(x['added_time'], x['parsed_at']), axis=1)

In [14]:
df.added_time

0      2020-03-30 23:06:00
1      2020-03-30 18:39:00
2      2020-03-30 18:09:00
3      2020-03-30 15:10:00
4      2020-03-30 22:57:00
               ...        
1947   2020-03-05 17:53:00
1948   2020-03-02 19:13:00
1949   2020-03-02 10:47:00
1950   2020-03-12 15:45:00
1951   2020-03-01 21:28:00
Name: added_time, Length: 1939, dtype: datetime64[ns]

# Получим координаты места по адресу через Геокодер Яндекса

In [15]:
import requests
import json
from secret import API_KEY
example = df.iloc[2].address
example

'Санкт-Петербург, Калининский район, муниципальный округ Академическое'

In [16]:
request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, example.replace(' ', '+'))
r = requests.get(request)
response = json.loads(r.text)
response

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'request': 'Санкт-Петербург, Калининский район, муниципальный округ Академическое',
     'results': '10',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'other',
        'text': 'Россия, Санкт-Петербург, Калининский район, муниципальный округ Академическое',
        'kind': 'district',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Санкт-Петербург, Калининский район, муниципальный округ Академическое',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Северо-Западный федеральный округ'},
          {'kind': 'province', 'name': 'Санкт-Петербург'},
          {'kind': 'locality', 'name': 'Санкт-Петербург'},
          {'kind': 'district', 'name': 'Калининский район'},
          {'kind': 'district', 'name': 'муниципальный округ Академическое'}]},
        'AddressD

In [17]:
def get_coords(x):
    request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
    r = requests.get(request)
    coords = json.loads(r.text)['response']
    coords = coords['GeoObjectCollection']['featureMember'][0]
    coords = coords['GeoObject']['Point']['pos']
    coords = coords.split()[::-1]
    coords = list(map(float, coords))
    return coords

In [18]:
get_coords(example)

[60.01691, 30.385721]

In [19]:
df['coords'] = df.address.map(get_coords)

"Растащим" список на две отдельные колонки

In [20]:
df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
df = df.drop('coords', 1)

Соберем это в отдельную функцию

In [21]:
def geocode(df):
    def get_coords(x):
        request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
        r = requests.get(request)
        coords = json.loads(r.text)['response']
        coords = coords['GeoObjectCollection']['featureMember'][0]
        coords = coords['GeoObject']['Point']['pos']
        coords = coords.split()[::-1]
        coords = list(map(float, coords))
        return coords
    
    df['coords'] = df.address.map(get_coords)
    df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
    df = df.drop('coords', 1)
    return df